#  normalize NOAA DSDP taxa list
## 101-210 taxa

Cleanup the normalized taxa list from the eODP researchers.

In [1]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [2]:
date='2023-06-26'
# date='2022-12-01'
old_species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_101_210_{date}.csv'
old_genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_101_210_{date}.csv'
old_higher_path = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_101_210_{date}.csv'



date = '2024-01-18'
# date='2023-06-26'
initial_input_file = RAW_DATA_DIR/'PI_processed_files'/f'NOAA_2_taxa_lists_taxa_list_{date}.csv'
input_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"
species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_101_210_{date}.csv'
genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_101_210_{date}.csv'
higher_path = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_101_210_{date}.csv'




## fix incorect pbdb_taxon_id

incorported  pbdb_taxon_id that the PIs corrected into the taxalist.


In [44]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)
# 8906

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (temp; JAS will remove when finished with dinos)',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'Corrections to pbdb_taxon',
       'species_taxon_id', 'species_taxon_name', 'genus_taxon_id',
       'genus_taxon_name', 'family_taxon_id', 'family_taxon_name',
       'order_taxon_id', 'order_taxon_name', 'subclass_taxon_id',
       'subclass_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [46]:
cols = [
    'Corrections to pbdb_taxon',
    'pbdb_taxon_id'
]
tmp = PI_df[PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon,pbdb_taxon_id
1084,414350,NaN
1622,434997,434997
1836,477767,8709
4844,71247,NaN
6056,418970,NaN
6202,430646,NaN
7237,1468,NaN


In [47]:
correct_col = 'Corrections to pbdb_taxon'
# pbdb.fix_pbdb_id(PI_df, '445341', 445341, correct_col)
# pbdb.fix_pbdb_id(PI_df, '414258', 414258, correct_col)
# pbdb.fix_pbdb_id(PI_df, '468292', 468292, correct_col)
# pbdb.fix_pbdb_id(PI_df, '434997', 434997, correct_col)

pbdb.fix_pbdb_id(PI_df, '414350', 414350, correct_col)
pbdb.fix_pbdb_id(PI_df, '477767', 477767, correct_col)
pbdb.fix_pbdb_id(PI_df, '71247', 71247, correct_col)
pbdb.fix_pbdb_id(PI_df, '418970', 418970, correct_col)
pbdb.fix_pbdb_id(PI_df, '430646', 430646, correct_col)
pbdb.fix_pbdb_id(PI_df, '1468', 1468, correct_col)

414350
477767
71247
418970
430646
1468


In [48]:
tmp = PI_df[PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon,pbdb_taxon_id
1084,414350,414350
1622,434997,434997
1836,477767,477767
4844,71247,71247
6056,418970,418970
6202,430646,430646
7237,1468,1468


In [49]:
del PI_df['corrected']

In [50]:
PI_df.shape

(8906, 40)

In [51]:
PI_df.to_csv(input_file, index=False)


## update species list

PIs changed the taxa names for many taxa that did not have pbdb data; search pbdb for these revised taxa names

In [99]:
old_species_df = pd.read_csv(old_species_path, dtype=str)

log_df(old_species_df)
# (5346, 19)

(5346, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
old_species_df.loc[old_species_df['pbdb_taxon_id'].isna(), 'check'] = False
old_species_df.loc[old_species_df['pbdb_taxon_id'].notna(), 'check'] = True

del old_species_df['taxon_group']

log_df(old_species_df)


(5346, 19)


,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,check
0,Aandalusiella ivoirensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [101]:
PI_df = pd.read_csv(input_file, dtype=str )

log_df(PI_df)
# (8906, 40)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
pbdb.add_genus_species(PI_df)
log_df(PI_df)

(8906, 41)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus species
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN,Andalusiella ivoirensis
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abratopdinium cardioforme


In [103]:
species_df = PI_df[PI_df['genus species'].notna()].copy()[['taxon_group',  'genus species']]
species_df.drop_duplicates(inplace=True)

log_df(species_df)
# (5220, 2)

(5220, 2)


,taxon_group,genus species
3,dinoflagellates,Andalusiella ivoirensis
4,dinoflagellates,Abratopdinium cardioforme
5,dinoflagellates,Abratopdinium kerguelense
6,dinoflagellates,Acanthaulax granulata
7,dinoflagellates,Acanthaulax wilsonii


In [104]:
merge_df = species_df.merge(old_species_df, how='left', on='genus species', 
                             indicator=True)

log_df(merge_df)
# (5411, 21)


(5411, 21)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,check,_merge
0,dinoflagellates,Andalusiella ivoirensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,both
2,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,both
3,dinoflagellates,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,both
4,dinoflagellates,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,both


In [105]:
merge_df['_merge'].value_counts()

both          5092
left_only      319
right_only       0
Name: _merge, dtype: int64

In [107]:
print(merge_df[merge_df['pbdb_taxon_id'].notna()].shape)
print(merge_df[merge_df['pbdb_taxon_id'].isna()].shape)

(1057, 20)
(4354, 20)


In [116]:
pbdb.fetch_pdbd_data(merge_df, 'genus species')

5250 5300 5400 

In [118]:
del merge_df['_merge']
del merge_df['check']

log_df(merge_df)

(5411, 21)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,species_taxon_id,species_taxon_name
0,dinoflagellates,Andalusiella ivoirensis,323958,Andalusiella ivoirensis,species,276893,Andalusiella,277915,Peridiniaceae,277919,...,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN,323958,Andalusiella ivoirensis
1,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
merge_df.to_csv(species_path, index=False)

## add species pbdb info to unapproved taxa 

In [120]:
species_df = pd.read_csv(species_path, dtype=str)

log_df(species_df)
# (5411, 22)

(5411, 21)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,species_taxon_id,species_taxon_name
0,dinoflagellates,Andalusiella ivoirensis,323958,Andalusiella ivoirensis,species,276893,Andalusiella,277915,Peridiniaceae,277919,...,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN,323958,Andalusiella ivoirensis
1,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
filter_species_df = species_df[species_df['pbdb_taxon_name'].notna() & (species_df['pbdb_taxon_rank'].isin(['species', 'subspecies']))]
log_df(filter_species_df)

#  (1169, 21)

(1169, 21)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,species_taxon_id,species_taxon_name
0,dinoflagellates,Andalusiella ivoirensis,323958,Andalusiella ivoirensis,species,276893,Andalusiella,277915,Peridiniaceae,277919,...,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN,323958,Andalusiella ivoirensis
5,dinoflagellates,Achilleodinium bianii,323992,Achilleodinium bianii,species,323991,Achilleodinium,321603,Gonyaulacaceae,321606,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,dinoflagellates,Achomosphaera ramulifera,277049,Achomosphaera ramulifera,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,dinoflagellates,Achomosphaera sagena,323552,Achomosphaera sagena,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,dinoflagellates,Achomosphaera triangulata,323940,Achomosphaera triangulata,species,277048,Achomosphaera,321603,Gonyaulacaceae,321606,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
PI_df = pd.read_csv(input_file, dtype=str)
pbdb.add_genus_species(PI_df)

log_df(PI_df)
# (8906, 41)

(8906, 41)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus species
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN,Andalusiella ivoirensis
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abratopdinium cardioforme


In [127]:
pbdb.add_pbdb_data(PI_df, species_df, 'genus species')
del PI_df['genus species']

In [128]:
 set(species_df.columns) - set(PI_df.columns)

{'genus species'}

In [129]:
PI_df.to_csv(input_file, index=False)

## update genera csv

In [179]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
old_genus_df =  pd.read_csv(old_genus_path, dtype=str)

log_df(old_genus_df)

(2313, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
old_genus_df.loc[old_genus_df['pbdb_taxon_id'].isna(), 'check'] = False
old_genus_df.loc[old_genus_df['pbdb_taxon_id'].notna(), 'check'] = True


del old_genus_df['taxon_group'] 
old_genus_df.drop_duplicates(inplace=True)


log_df(old_genus_df)

(2255, 21)


,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,family_taxon_id,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,277915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,277915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aandalusiella,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abratopdinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
genus_df = df[df['genus name'].notna()].copy()[['taxon_group', 'genus name']]
genus_df.drop_duplicates(inplace=True)

log_df(genus_df)

(2067, 2)


,taxon_group,genus name
0,dinoflagellates,Labyrinthodinium
1,dinoflagellates,Maduradinium
2,dinoflagellates,Pyxidiella
3,dinoflagellates,Andalusiella
4,dinoflagellates,Abratopdinium


In [183]:
merge_df = genus_df.merge(old_genus_df, how='left', on='genus name',
                         indicator=True, suffixes=(None, '_y'))


log_df(merge_df)
# (2067, 23)

(2067, 23)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,dinoflagellates,Andalusiella,True,276893,Andalusiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,dinoflagellates,Abratopdinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


check for duplicate genus

In [184]:
merge_df[merge_df.duplicated(subset=['taxon_group', 'genus name'])]

,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge


In [185]:
pbdb.fetch_pdbd_data(merge_df, 'genus name')


200 800 1200 1250 1400 1950 2000 

In [186]:
log_df(merge_df)

(2067, 23)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,dinoflagellates,Andalusiella,True,276893,Andalusiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,dinoflagellates,Abratopdinium,True,477210,Abratopdinium,genus,277919,Peridiniales,321578,Dinophyceae,...,Myzozoa,NaN,NaN,NaN,NaN,NaN,NaN,477210,Abratopdinium,both


In [187]:
print(merge_df[merge_df['pbdb_taxon_id'].isna()].shape)
print(merge_df[merge_df['pbdb_taxon_id'].notna()].shape)

(151, 23)
(1916, 23)


In [188]:
merge_df.drop_duplicates(inplace=True)

log_df(merge_df)

(2067, 23)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,dinoflagellates,Andalusiella,True,276893,Andalusiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,dinoflagellates,Abratopdinium,True,477210,Abratopdinium,genus,277919,Peridiniales,321578,Dinophyceae,...,Myzozoa,NaN,NaN,NaN,NaN,NaN,NaN,477210,Abratopdinium,both


In [189]:
merge_df.to_csv(genus_path, index=False)

## update genus csv with no higher pbdb ranks

some genus taxa don't have pbdb info for higher taxa ranks;  add missing pbdb data

In [190]:
df = pd.read_csv(genus_path, dtype=str)

df.columns

Index(['taxon_group', 'genus name', 'check', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'order_taxon_id',
       'order_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'family_taxon_id', 'family_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'subclass_taxon_id', 'subclass_taxon_name', 'genus_taxon_id',
       'genus_taxon_name', '_merge'],
      dtype='object')

In [ ]:
temp_df = df[
    df['unranked clade_taxon_id'].isna() &
    df['kingdom_taxon_id'].isna() &
    df['phylum_taxon_id'].isna() &
    df['family_taxon_id'].isna() &
    df['order_taxon_id'].isna() &
    df['class_taxon_id'].isna() &
    df['subclass_taxon_id'].isna() &
    df['pbdb_taxon_id'].notna() 
].copy()
log_df(temp_df)

(33, 23)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge
5,dinoflagellates,Acanthaulax,True,276986,Acanthaulax,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
15,dinoflagellates,Alterbidinium,True,276891,Alterbidinium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
28,dinoflagellates,Artemisiocysta,True,477230,Artemisiocysta,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,477230,Artemisiocysta,both
34,dinoflagellates,Bachmayerella,True,477233,Bachmayerella,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,477233,Bachmayerella,both
47,dinoflagellates,Caligodinium,True,276897,Caligodinium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [197]:
for index, row in temp_df.iterrows():          
    time.sleep(0.5)
        
    url =  pbdb.PBDB_TAXA_LIST_ID + str(row['pbdb_taxon_id'])
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
 
        if len(data) > 0:
            last_record = data[len(data) - 1]
            df.at[index, 'pbdb_taxon_id'] = last_record["oid"].replace('txn:', '')
            df.at[index, 'pbdb_taxon_name'] = last_record["nam"]
            df.at[index, 'pbdb_taxon_rank'] = pbdb.rank_ids[last_record["rnk"]]


            pbdb.process_taxa_hierarchy(df, data, index)
    else:
        print(name, ' not found')

In [199]:
df.to_csv(genus_path, index=False)

## add genus pbdb to input_file

In [207]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)


(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
genus_df= pd.read_csv(genus_path, dtype=str)

log_df(genus_df)
# (2067, 23)

(2067, 23)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,_merge
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,dinoflagellates,Andalusiella,True,276893,Andalusiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,dinoflagellates,Abratopdinium,True,477210,Abratopdinium,genus,277919,Peridiniales,321578,Dinophyceae,...,Myzozoa,NaN,NaN,NaN,NaN,NaN,NaN,477210,Abratopdinium,both


In [209]:
pbdb.add_pbdb_data(df, genus_df, 'genus name')

In [210]:
log_df(df)
# (8906, 40)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN


In [211]:
df.to_csv(input_file, index=False)

## create higher taxa csv 

In [212]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN


check for for taxon that has both higher taxa and genus name 

In [213]:
df[df['Any taxon above genus'].notna() & df['genus name'].notna()]

,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
6087,nannofossils,M. ondosus,"""M. ondosus""",NaN,From Leah: TLDR: I don't know what to do with ...,NaN,NaN,"""M. ondosus""",NaN,M.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28595,Life


In [214]:
higher_df =  pbdb.create_higher_taxa_df(df)
higher_df['check'] = False
log_df(higher_df)
# (162, 16)

(162, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
31,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
376,dinoflagellates,"""Cyst type 1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
377,dinoflagellates,"""Cyst type A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
416,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
703,dinoflagellates,"""Incertae sedis forma A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [215]:
higher_df[higher_df.duplicated(subset=['taxon_group', 'Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
4531,benthic_forams,Textulariidae indet.,2661,Textularia,genus,82194,Textulariidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,False
7098,ostracods,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
8517,radiolarians,Plagoniidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Radiolaria,212476,Rhizaria,False


In [216]:
higher_df[higher_df.duplicated(subset=['Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
1349,palynology,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4531,benthic_forams,Textulariidae indet.,2661,Textularia,genus,82194,Textulariidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,False
6872,ostracods,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7098,ostracods,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
7608,planktic_forams,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8329,radiolarians,Radiolaria indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8517,radiolarians,Plagoniidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Radiolaria,212476,Rhizaria,False
8891,radiolarians,"""Sponge spicules""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [218]:
higher_df[higher_df.duplicated()]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check


In [219]:
pbdb.fetch_pdbd_data(higher_df, 'Any taxon above genus')

3250 "Azpeitia aff. Thalassiosira bukryi; Thalassiosira"  not found
"other Freshwater (Achnanthes; Fragilaria, etc.)"  not found


In [220]:
higher_df.drop_duplicates(inplace=True)

log_df(higher_df)


(162, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
31,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
376,dinoflagellates,"""Cyst type 1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
377,dinoflagellates,"""Cyst type A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
416,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
703,dinoflagellates,"""Incertae sedis forma A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [221]:
higher_df.columns

Index(['taxon_group', 'Any taxon above genus', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name', 'check'],
      dtype='object')

In [222]:
higher_df.to_csv(higher_path, index=False)

## add higher taxa pbdb to input file

In [223]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN


In [225]:
higher_df = pd.read_csv(higher_path, dtype=str)
log_df(higher_df)
# (162, 16)

(162, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
0,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,dinoflagellates,"""Cyst type 1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,dinoflagellates,"""Cyst type A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
4,dinoflagellates,"""Incertae sedis forma A""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [226]:
pbdb.add_pbdb_data(df, higher_df, 'Any taxon above genus')

In [227]:
log_df(df)

(8906, 40)


,taxon_group,verbatim_name,name,Comment,Notes (temp; JAS will remove when finished with dinos),Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,Andalusiella ivoirensis,genus missplled,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Andalusiella,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,321578,Dinophyceae,277918,Myzozoa,NaN,NaN,NaN,NaN


In [228]:
df.to_csv(input_file, index=False)

## QA taxa

In [201]:
df = pd.read_csv(input_file, dtype=str, nrows=2441)
log_df(df)

(2441, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
filter_df = df[df['pbdb_taxon_id'].isna()]
log_df(filter_df)

(258, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,31,dinoflagellates,Acritarch forma A,Acritarcha indet.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,Acritarcha indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,32,dinoflagellates,Acritarch forma B,Acritarcha indet.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,Acritarcha indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
genus_df = filter_df[['genus name']].copy()
genus_df.drop_duplicates(inplace=True)
genus_df.dropna(inplace=True)
genus_df['check'] = False
log_df(genus_df)

(103, 2)


,genus name,check
3,Aandalusiella,False
4,Abratopdinium,False
40,Aireiana,False
44,Algidasphaeridium,False
61,Alisogymnium,False


In [214]:
pbdb.fetch_pdbd_data(genus_df, 'genus name')

500 

In [215]:
log_df(genus_df)

(103, 3)


,genus name,check,pbdb_taxon_id
3,Aandalusiella,True,<NA>
4,Abratopdinium,True,<NA>
40,Aireiana,True,<NA>
44,Algidasphaeridium,True,<NA>
61,Alisogymnium,True,<NA>


## reorder columns

In [219]:
df = pd.read_csv(input_file, dtype=str)
del df['Unnamed: 0']
log_df(df)

(8906, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)

(8906, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
PI_df.columns == df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

### ========================================
## stop
### ========================================


## create crosswalk file

In [96]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [97]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [98]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
log_df(crosswalk_df)
# 7763
# 7760
# 7759

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7760, 17)
drop duplicates df:  (7759, 17)
(7759, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,


In [99]:
PI_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [100]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [101]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [102]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7760
# 7658

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7760, 25)
drop duplicates df:  (7658, 25)
(7658, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [103]:
taxa_df.to_csv(taxa_file, index=False)